In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
data = pd.read_excel("URINELOTS.xlsx")
data = data[['UHID','RAMJA Results','AIIMS Result culture']]
data.columns = ['UHID','RAMJA','AIIMS']
data.head()

,UHID,RAMJA,AIIMS
0,20220065432,Enterococcus faecalis,Enterococcus faecalis
1,20240016182,NaN,Contamination
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION
4,20201415102,NaN,NaN


In [4]:
data.shape

(146, 3)

In [5]:
#df_nan.to_excel('compareddata.xlsx', sheet_name='Null data')

In [6]:
import re
def cleanup(text):
    text = str(text)
    text = text.upper()
    text = text.replace(".", " ")
    text = text.replace(" AND ", " ")
    text = re.sub(r'\s+', ' ', text)  # Use re.sub for regular expression replacement
    text = text.strip()
    return text


In [7]:
bacteriatypes = {
    "COLI" : 'Escherichia coli', 
    "PNEUMO" : 'Klebsiella pneumoniae',
    "AERUGI" : 'Pseudomonas aeruginosa',
    "BAUM" : 'Acinetobacter baumannii', 
    "MIRABIL" : 'Proteus mirabilis', 
    "FAECALI" : 'Enterococcus faecalis', 
    "AUREUS" : 'Staphylococcus aureus', 
    "SAPROPHY" : 'Staphylococcus saprophyticus',
    "STERI" : 'Sterile',
    "CONTA" : 'Contamination',
    "NAN" : 'NAN'
}

In [8]:
data['cleanRAMJA'] = data['RAMJA'].apply(cleanup)
data['cleanAIIMS'] = data['AIIMS'].apply(cleanup)

In [9]:
def check(mainstring, s1, s2):
    if (mainstring.count(s1) > 0 or mainstring.count(s2) > 0):
        #print("YES")
        return(1)
    else:
        #print("NO")
        return(0)

### RAMJA Analysis

In [10]:
df_R = data.iloc[:,0:6]
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI","CONTA","NAN"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUM", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI","CONTA","NAN"]
for s1, s2 in zip(genus,species):
    df_R[s2] = data['cleanRAMJA'].apply(lambda x: check(x, s1, s2))

df_R.rename(columns=bacteriatypes, inplace=True) #change column names
# add suffix _R to column names
# columns_to_modify = ['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile','Contamination','NAN']
# data.rename(columns={col: f"{col}_R" for col in columns_to_modify}, inplace=True)

In [11]:
df_R.shape

(146, 16)

In [12]:
#....add a column named 'Others' contain values other than assigend catagories
df_R_temp = df_R[['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile','Contamination','NAN']]
df_R_temp = df_R_temp == 0
df_R_temp['Others_R'] = df_R_temp.all(axis=1)
df_R['Others_R'] = df_R_temp['Others_R'].astype(int)
df_R.shape

(146, 17)

In [13]:
#...Get infections positive column in RAMJA
df_R_temp  = df_R[['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus']]
df_R_temp = df_R_temp == 1
df_R_temp['Infection_R'] = df_R_temp.any(axis=1)
df_R['Infection_R'] = df_R_temp['Infection_R'].astype(int)
df_R.shape

(146, 18)

In [14]:
#....Drop rows containing NaN in Ramaj column results
df_NonNaN_R = df_R.dropna(subset=['RAMJA'])
df_NonNaN_R.shape

(143, 18)

In [15]:
#...Remove contaminated sample and Others samples
df_R_clean = df_NonNaN_R[(df_NonNaN_R['Contamination'] == 0) & (df_NonNaN_R['Others_R'] == 0)]
df_R_clean.shape

(143, 18)

In [16]:
#....After Dopping NaN these two set of values should be equal
print(df_NonNaN_R['Infection_R'].value_counts())
print(df_NonNaN_R['Sterile'].value_counts())

1    97
0    46
Name: Infection_R, dtype: int64
0    97
1    46
Name: Sterile, dtype: int64


In [17]:
#df_R.to_excel("temp1.xlsx")

### AIIMS Analysis

In [18]:
df_A = data.iloc[:,0:6]
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI","CONTA","NAN"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUM", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI","CONTA","NAN"]
for s1, s2 in zip(genus,species):
    df_A[s2] = data['cleanAIIMS'].apply(lambda x: check(x, s1, s2))

df_A.rename(columns=bacteriatypes, inplace=True) #change column names

# add suffix _R to column names
#columns_to_modify = ['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile','Contamination','NAN']
#data.rename(columns={col: f"{col}_A" for col in columns_to_modify}, inplace=True)

In [19]:
#....add a column named 'Others' contain values other than assigend catagories
df_A_temp = df_A[['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile','Contamination','NAN']]
df_A_temp = df_A_temp == 0
df_A_temp['Others_A'] = df_A_temp.all(axis=1)
df_A['Others_A'] = df_A_temp['Others_A'].astype(int)
df_A.shape

(146, 17)

In [20]:
#...Get infections positive (any infection) in AIIMS
df_A_temp  = df_A[['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus']]
df_A_temp >= 1
df_A_temp['Infection_A'] = df_A_temp.any(axis=1)
df_A['Infection_A'] = df_A_temp['Infection_A'].astype(int)
df_A.shape


C:\Users\neelesh\AppData\Local\Temp\ipykernel_9692\844663831.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_A_temp['Infection_A'] = df_A_temp.any(axis=1)


(146, 18)

In [21]:
#...dataset containing removed NaN based on AIIMS
df_NonNaN_A = df_A.dropna(subset=['AIIMS'])

In [22]:
#...Remove contaminated sample and Others samples 
df_A_clean = df_NonNaN_A[(df_NonNaN_A['Contamination'] == 0) & (df_NonNaN_A['Others_A'] == 0)]
df_A_clean.shape

(94, 18)

In [23]:
#....After Dopping NaN these two set of values should be equal
print(df_A_clean['Infection_A'].value_counts())
print(df_A_clean['Sterile'].value_counts())

0    64
1    30
Name: Infection_A, dtype: int64
1    64
0    30
Name: Sterile, dtype: int64


In [24]:
#df_NonNaN_A.to_excel("temp1.xlsx")

In [25]:
#......
df_R_infec = df_R_clean[df_R_clean['Infection_R'] == 1]
df_R_infec = df_R_infec[['UHID','RAMJA','AIIMS']]
df_R_infec.shape

(97, 3)

In [26]:
df_A_infec = df_A_clean[df_A_clean['Infection_A'] == 1]
df_A_infec = df_A_infec[['UHID','RAMJA','AIIMS']]
df_A_infec.shape

(30, 3)

### Concordance analysis

In [27]:
#...Concordance: Get same type of infections in both RAMJA and AIIMS
df_R_temp1  = df_R[['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus']]
df_A_temp1  = df_A[['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus']]
df_C_temp1 = df_R_temp1 & df_A_temp1
df_C_temp1['Infection_C'] = df_C_temp1.any(axis=1)
df_C = data[['UHID','RAMJA','AIIMS']]
df_C['concordance'] = df_C_temp1['Infection_C'].astype(int)
df_C.head()

C:\Users\neelesh\AppData\Local\Temp\ipykernel_9692\3825133095.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_C['concordance'] = df_C_temp1['Infection_C'].astype(int)


,UHID,RAMJA,AIIMS,concordance
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,1
1,20240016182,NaN,Contamination,0
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,1
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,0
4,20201415102,NaN,NaN,0


In [28]:
df_C['concordance'].value_counts()

0    128
1     18
Name: concordance, dtype: int64

### common clean samples

In [30]:
df_R_tempc = df_R_clean[['UHID','RAMJA','AIIMS', 'Infection_R']]
df_R_tempc.shape

(143, 4)

In [31]:
df_A_tempc = df_A_clean[['UHID','Infection_A']]
df_A_tempc.shape

(94, 2)

In [32]:
df_C_clean = pd.merge(df_R_tempc,df_A_tempc,how='inner', on='UHID')
df_C_clean.shape

(94, 5)

In [33]:
df_C_clean.head()

,UHID,RAMJA,AIIMS,Infection_R,Infection_A
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,1,1
1,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,1,1
2,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...",E.coli,1,1
3,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus",E.coli & Pseudomnas,1,1
4,20230149608,"E. Coli, Pseudomonas, P.mirabilis, E.Faecalis,...","E.coli,A.Baumini",1,1


In [76]:
from sklearn.metrics import confusion_matrix

y_true = df_C_clean['Infection_A']
y_pred = df_C_clean['Infection_R']

cm = confusion_matrix(y_true, y_pred)
print(cm)

[[26 38]
 [ 4 26]]


In [79]:
sum(sum(cm))

94

In [35]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

In [36]:
fp

38

In [77]:
t_cm = np.array([tp, fp, fn, tn]).reshape(2,2)
t_cm

array([[26, 38],
       [ 4, 26]], dtype=int64)

In [38]:
def scores(y_test, y_pred):
        #y_pred = self.fit(x_train, x_test, y_train)
        from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
        
        cm = confusion_matrix(y_test,y_pred)
        
        # Compute confusion matrix
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
                
        #.....................Accuracy......................#
        accur = accuracy_score(y_test,y_pred)
        
       #.....................Senstivity.....................#
        sens = tp/(tp + fn)

        #.....................Specificity...................#
        spec = tn / (tn + fp)

        #..........Positive predictive value (PPV)...........#
        PPV = tp/(tp + fp)

        #..........negative predictive value (NPV)...........#
        NPV = tn/(tn + fn)
        
        #..........Positive Likelihood ratio (PLR)...........#
        PLR = sens/(1-spec)

        #..........Negative Likelihood ratio (NLR)...........#
        NLR = (1-sens)/spec

        errorList = np.array([accur, sens, spec, PPV, NPV, PLR, NLR])
        return(errorList)

In [39]:
scores(y_true,y_pred)

array([0.55319149, 0.86666667, 0.40625   , 0.40625   , 0.86666667,
       1.45964912, 0.32820513])

In [55]:
scoreid = ['Accuracy', 'Sensitivity', 'Specificity', 'PPV', 'NPV', 'PLR', 'NLR']

In [46]:
df_score = pd.DataFrame(scores(y_true,y_pred),index=scoreid, columns= ['LOT1'])

In [60]:
df_score = round(df_score,4)
df_score

,LOT1
Accuracy,0.5532
Sensitivity,0.8667
Specificity,0.4062
PPV,0.4062
NPV,0.8667
PLR,1.4596
NLR,0.3282


In [48]:
df_cm =pd.DataFrame(t_cm,index=['RAMJA Positive','RAMJA Negative'], columns= ['AIIMS Positive','AIIMS Negative'])
df_cm

,AIIMS Positive,AIIMS Negative
RAMJA Positive,26,38
RAMJA Negative,4,26


In [49]:
""" print("#..........RAMJA Details..........#")
#......VAlies for both RAMJA and AIIMS
print("Total samples: ", data.shape[0])

#...Values for RAMJA
print("Null Samples RAMJA: ",data['RAMJA'].isnull().sum(axis=0))
print("Non-Null RAMJA: ", data['RAMJA'].count())

#...VAlues for RAMJA after dropping NaN
print("Contaminated Samples RAMJA: ", df_NonNaN_R['Contamination'].sum())
print("Other RAMJA: ", df_NonNaN_R['Others_R'].sum())
print("Positive RAMJA: ", df_NonNaN_R['Infection_R'].sum())
print("Negative RAMJA: ", df_NonNaN_R['Sterile'].sum())

print("#..........AIIMS Details..........#")
#...Values for AIIMS
print("Total samples: ", data.shape[0])
print("Null Samples AIIMS: ",data['AIIMS'].isnull().sum(axis=0))
print("Non-Null AIIMS: ", data['AIIMS'].count())

#...VAlues for RAMJA after dropping NaN
print("Contaminated Samples AIIMS: ", df_NonNaN_A['Contamination'].sum())
print("Other AIIMS: ", df_NonNaN_A['Others_A'].sum())
print("Positive AIIMS: ", df_NonNaN_A['Infection_A'].sum())
print("Negative AIIMS: ", df_NonNaN_A['Sterile'].sum())
 """

' print("#..........RAMJA Details..........#")\n#......VAlies for both RAMJA and AIIMS\nprint("Total samples: ", data.shape[0])\n\n#...Values for RAMJA\nprint("Null Samples RAMJA: ",data[\'RAMJA\'].isnull().sum(axis=0))\nprint("Non-Null RAMJA: ", data[\'RAMJA\'].count())\n\n#...VAlues for RAMJA after dropping NaN\nprint("Contaminated Samples RAMJA: ", df_NonNaN_R[\'Contamination\'].sum())\nprint("Other RAMJA: ", df_NonNaN_R[\'Others_R\'].sum())\nprint("Positive RAMJA: ", df_NonNaN_R[\'Infection_R\'].sum())\nprint("Negative RAMJA: ", df_NonNaN_R[\'Sterile\'].sum())\n\nprint("#..........AIIMS Details..........#")\n#...Values for AIIMS\nprint("Total samples: ", data.shape[0])\nprint("Null Samples AIIMS: ",data[\'AIIMS\'].isnull().sum(axis=0))\nprint("Non-Null AIIMS: ", data[\'AIIMS\'].count())\n\n#...VAlues for RAMJA after dropping NaN\nprint("Contaminated Samples AIIMS: ", df_NonNaN_A[\'Contamination\'].sum())\nprint("Other AIIMS: ", df_NonNaN_A[\'Others_A\'].sum())\nprint("Positive AIIM

In [50]:
samplesDetail = {
    "Total Samples": [data.shape[0],data.shape[0]],
    "Null Samples": [data['RAMJA'].isnull().sum(axis=0), data['AIIMS'].isnull().sum(axis=0)],
    "Non-Null": [data['RAMJA'].count(),data['AIIMS'].count()],
    "Contaminated Samples": [df_NonNaN_R['Contamination'].sum(),df_NonNaN_A['Contamination'].sum()],
    "Others [Not defined]": [df_NonNaN_R['Others_R'].sum(),df_NonNaN_A['Others_A'].sum()],
    "Useful Samples":[df_R_clean.shape[0], df_A_clean.shape[0]],
    "Positive": [df_NonNaN_R['Infection_R'].sum(),df_NonNaN_A['Infection_A'].sum()],
    "Negative": [df_NonNaN_R['Sterile'].sum(),df_NonNaN_A['Sterile'].sum()],
    "Common Positive": [tp,tp],
    "Concordance [Common bacteria]": [df_C['concordance'].sum(),df_C['concordance'].sum()],
    "Common Negative": [tn,tn]
    
}

In [51]:
df_samplesDetail = pd.DataFrame(samplesDetail, index=['RAMJA','AIIMS']).T
df_samplesDetail

,RAMJA,AIIMS
Total Samples,146,146
Null Samples,3,27
Non-Null,143,119
Contaminated Samples,0,14
Others [Not defined],0,11
Useful Samples,143,94
Positive,97,30
Negative,46,64
Common Positive,26,26
Concordance [Common bacteria],18,18


In [61]:
# Write the DataFrames to an Excel file
with pd.ExcelWriter('matrices.xlsx') as writer:
    df_cm.to_excel(writer, sheet_name='SamplesDetail', index=True)
    df_samplesDetail.to_excel(writer, sheet_name='ConfusionMatrix', index=True)
    df_score.to_excel(writer, sheet_name='Scores', index=True)
    


In [72]:
df_A_conta = df_NonNaN_A[df_NonNaN_A['Contamination'] ==1][['UHID', 'RAMJA', 'AIIMS']]
df_A_conta.shape

(14, 3)

In [73]:
df_A_others = df_NonNaN_A[df_NonNaN_A['Others_A'] ==1][['UHID', 'RAMJA', 'AIIMS']]
df_A_others.shape

(11, 3)

In [86]:
#...Both positives
df_Pos_RA = df_C_clean[(df_C_clean['Infection_R'] == 1) & (df_C_clean['Infection_A'] == 1)][['UHID', 'RAMJA', 'AIIMS']]
df_Pos_RA.shape

(26, 3)

In [85]:
#...Both Negatives
df_Neg_RA = df_C_clean[(df_C_clean['Infection_R'] == 0) & (df_C_clean['Infection_A']==0)][['UHID', 'RAMJA', 'AIIMS']]
df_Neg_RA.shape

(26, 3)

In [82]:
#...Common type of infection
df_concordance = df_C[df_C['concordance'] == 1][['UHID','RAMJA','AIIMS']]
df_concordance.shape

(18, 3)

In [ ]:
# Write the DataFrames to an Excel file
with pd.ExcelWriter('sorteddata.xlsx') as writer:
    df_A_others.to_excel(writer, sheet_name='Undefined Samples_A', index=False)
    df_A_conta.to_excel(writer, sheet_name='Contaminated Samples_A', index=False)
    df_Pos_RA.to_excel(writer, sheet_name='Common Positives', index=False)
    df_concordance.to_excel(writer, sheet_name='Concordance', index=False)
    df_Neg_RA.to_excel(writer, sheet_name='Common Negatives', index=False)

### Confusion matrix bwtween RAMJA and AIIMS

In [84]:
df_infection = df_infection.dropna(subset=['RAMJA','AIIMS'])

In [86]:
df_infection.head()

,UHID,RAMJA,AIIMS,Infection_A,Infection_R
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,1,1
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,1,1
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,0,1
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...",E.coli,1,1
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus",E.coli & Pseudomnas,1,1


In [88]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_infection['Infection_A'],df_infection['Infection_R'])
cm


array([[34, 54],
       [ 7, 23]], dtype=int64)

In [90]:
print(df_infection['Infection_A'].value_counts())
print(df_infection['Infection_R'].value_counts())

0    88
1    30
Name: Infection_A, dtype: int64
1    77
0    41
Name: Infection_R, dtype: int64


In [205]:
index = ['TN', 'FP', 'FN', 'TP']
columns = ['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile']
df_cm = pd.DataFrame(cm_dfs,index,columns)
df_cm

,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile
TN,68,88,103,93,91,88,90,107,42
FP,32,24,13,25,26,24,27,11,12
FN,7,4,0,0,0,3,1,0,38
TP,11,2,2,0,1,3,0,0,26


In [206]:
totalPositive = df_cm.loc['TP'] + df_cm.loc['FP']
totalNegative = df_cm.loc['TN'] + df_cm.loc['FN']
df_cm.loc['Total Positive'] = totalPositive
df_cm.loc['Total Negative'] = totalNegative

In [207]:
df_cm

,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile
TN,68,88,103,93,91,88,90,107,42
FP,32,24,13,25,26,24,27,11,12
FN,7,4,0,0,0,3,1,0,38
TP,11,2,2,0,1,3,0,0,26
Total Positive,43,26,15,25,27,27,27,11,38
Total Negative,75,92,103,93,91,91,91,107,80


In [211]:
aiims2 = [TotalSamples, aiimsNull, aiimsnonnull, common_nonnull, contaaiims]

In [212]:
aiims2

[146, 27, 119, 118, 14]